In [1]:
import tensorflow as tf
import numpy as np
import os
from simple import *
import math
from collections import defaultdict
import random

In [2]:
path = '../data/NER-RNN/ner'
train_path = os.path.join(path, 'eng.train')
testa_path = os.path.join(path, 'eng.testa')
testb_path = os.path.join(path, 'eng.testb')


In [3]:
def read_corpus(path):
    groups = [[]]
    for line in open(path):
        parts = line.strip().split()
        if len(parts):
            groups[-1].append((parts[0], parts[-1]))
        else:
            groups.append([])
    return groups

all_groups = flatten([read_corpus(f) for f in [train_path, testa_path, testb_path]])[1:]
all_tokens = flatten(all_groups)

entities = list(set([tk[1] for tk in all_tokens]))
chars = list(set(flatten([tk[0] for tk in all_tokens]))) + [' ']


In [30]:
char_lookup = {c: i+1 for i, c in enumerate(chars)}
entity_lookup = {e: i for i, e in enumerate(entities)}

def vectorize_sentence(tokens, bucket_rounding):
    seq_len = sum([len(tk) for tk, ent in tokens]) + len(tokens)
    seq_len = int(math.ceil(seq_len * 1.0 / bucket_rounding) * bucket_rounding)
    x = np.zeros(seq_len, dtype=int)
    y = np.zeros(seq_len, dtype=int)
    i = 0
    for token, ent in tokens:
        ent_idx = entity_lookup[ent]
        for c in token + ' ':
            x[i] = char_lookup[c]
            y[i] = ent_idx
            i += 1
    return x, y

def epoch(batch_size=16):
    sentences = all_groups[:]
    random.shuffle(sentences)
    buckets = defaultdict(list)
    for sentence in sentences:
        x, y = vectorize_sentence(sentence, 32)
        bucket = x.shape[0]
        buckets[bucket].append((x,y))
        if len(buckets[bucket]) == batch_size:
            xs = np.stack([x for x,y in buckets[bucket]])
            ys = np.stack([y for x,y in buckets[bucket]])
            buckets[bucket] = []
            yield xs, ys

# print all_groups[0]
# print vectorize_sentence(all_groups[0], 10)
batch_counts = {}
for i, (x, y) in enumerate(epoch(1)):
    batch_counts[x.shape[1]] = batch_counts.get(x.shape[1], 0) + 1
    if i > 1000: break

buckets = set([i*32 for i in xrange(1, 11)])

In [94]:

class NER(object):
    def __init__(self, buckets, n_chars, n_entities, scopename='ner'):
        with tf.variable_scope(scopename):
            pass
    
    def create_model(self, x):
        for i in xrange(4):
            # tf.layers.conv1d(inputs, filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, activation=None
            x = tf.layers.


RuntimeError: input and gradOutput shapes do not match: input [16 x 20 x 8], gradOutput [16 x 8 x 20] at /home/soumith/local/builder/wheel/pytorch-src/torch/lib/THNN/generic/LogSoftMax.c:78